In [101]:
# pip install openpyxl ffmpeg

In [102]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from IPython.display import HTML
from sklearn.svm import OneClassSVM
from scipy.interpolate import griddata
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib.animation import FuncAnimation, PillowWriter


In [103]:
data = pd.read_excel("Sensor data.xlsx",sheet_name=None)

In [104]:
data

{'0474D T|RH IP67(0474D-S1)':                 Sensor ID             0474D    Unnamed: 2
 0             Sensor Name   0474D T/RH IP67           NaN
 1            Sensor Group               NaN           NaN
 2                    Time  Temperature (°C)  Humidity (%)
 3     2024-04-16 13:09:53             21.95          50.3
 4     2024-04-16 13:14:52              22.5          34.7
 ...                   ...               ...           ...
 8358  2024-05-15 13:02:14              29.7            43
 8359  2024-05-15 13:07:13             29.55          44.9
 8360  2024-05-15 13:12:13              30.4          42.7
 8361  2024-05-15 13:17:13                31          44.2
 8362  2024-05-15 13:22:12              31.2          42.1
 
 [8363 rows x 3 columns],
 '04750 T|RH IP67(04750-S1)':                  Sensor ID             04750    Unnamed: 2
 0              Sensor Name   04750 T/RH IP67           NaN
 1             Sensor Group               NaN           NaN
 2                     Tim

In [105]:
def extract_data_from_sheet(df):
    sensor_name = df.iloc[0, 1]

    extracted_df = df.iloc[3:].reset_index(drop=True)

    extracted_df.columns = ['Time', 'Temperature', 'Humidity']
    extracted_df['Time'] = pd.to_datetime(extracted_df['Time'], format='%d-%m-%Y %H:%M')

    extracted_df['Sensor Info'] = sensor_name
    
    # Extracting the first 5 characters from 'Sensor Info' and creating 'Sensor ID' column
    extracted_df['Sensor ID'] = extracted_df['Sensor Info'].str[:5]

    return extracted_df

In [106]:
data_frames = []

for sheet_name, df in data.items():
    extracted_df = extract_data_from_sheet(df)
    data_frames.append(extracted_df)

In [107]:
combined_data = pd.concat(data_frames, ignore_index=True)

In [108]:
combined_data.head()

,Time,Temperature,Humidity,Sensor Info,Sensor ID
0,2024-04-16 13:09:53,21.95,50.3,0474D T/RH IP67,0474D
1,2024-04-16 13:14:52,22.5,34.7,0474D T/RH IP67,0474D
2,2024-04-16 13:19:54,24.05,34.7,0474D T/RH IP67,0474D
3,2024-04-16 13:24:53,24.25,34.7,0474D T/RH IP67,0474D
4,2024-04-16 13:29:52,24.15,35.2,0474D T/RH IP67,0474D


In [109]:
combined_data.isnull().sum()

Time              0
Temperature    2235
Humidity       2235
Sensor Info       0
Sensor ID         0
dtype: int64

In [110]:
# Check for null values
null_counts = combined_data.groupby('Sensor ID')[['Temperature', 'Humidity']].apply(lambda x: x.isnull().sum())

# Filter sensors with null values
sensors_with_nulls = null_counts[(null_counts['Temperature'] > 0) | (null_counts['Humidity'] > 0)]

print("Sensors with null values:")
print(sensors_with_nulls)

Sensors with null values:
           Temperature  Humidity
Sensor ID                       
0474D               25        25
04750               62        62
04754              313       313
04756              300       300
04757               20        20
0475A               23        23
0475C               51        51
0475D               52        52
04762               33        33
04763               54        54
04765              297       297
04768               54        54
0476E               77        77
04771              113       113
04773              290       290
0477F               40        40
04787              313       313
04789               60        60
0478A               26        26
0478C               32        32


In [111]:
sample_dataset = combined_data.copy()

In [115]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Function to convert Time to Unix timestamp and impute missing values using linear regression
def impute_missing_values(df, target_column, feature_columns):
    # Ensure Time column is in datetime format
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df['UnixTime'] = df['Time'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)  # Convert Time to Unix timestamp

    not_null_data = df.dropna(subset=[target_column])
    null_data = df[df[target_column].isnull()]

    if not_null_data.empty or null_data.empty:
        print(f"No valid data for regression in {target_column}.")
        return df

    X_train = not_null_data[feature_columns + ['UnixTime']].copy()
    y_train = not_null_data[target_column].values

    # Ensure no NaNs in X_train
    for col in feature_columns + ['UnixTime']:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    valid_indices = ~X_train.isnull().any(axis=1)
    X_train = X_train[valid_indices].values
    y_train = y_train[valid_indices]

    if X_train.size == 0:
        print(f"No valid training data for {target_column}.")
        return df

    X_predict = null_data[feature_columns + ['UnixTime']].copy()

    # Ensure no NaNs in X_predict
    for col in feature_columns + ['UnixTime']:
        X_predict[col] = pd.to_numeric(X_predict[col], errors='coerce')
    valid_predict_indices = ~X_predict.isnull().any(axis=1)
    X_predict = X_predict[valid_predict_indices].values
    null_indices = null_data.index[valid_predict_indices]

    if X_predict.size == 0:
        print(f"No valid prediction data for {target_column}.")
        return df

    model = LinearRegression()
    model.fit(X_train, y_train)

    predicted_values = model.predict(X_predict)
    df.loc[null_indices, target_column] = predicted_values

    return df.drop(columns=['UnixTime'])




In [116]:
# Example usage
# Assuming 'combined_data' is your dataframe and 'sensors_with_nulls' contains sensors with missing values

# Impute missing values for each sensor
for sensor_id in sensors_with_nulls.index:
    sensor_data = combined_data[combined_data['Sensor ID'] == sensor_id]
    
    if sensor_data['Temperature'].isnull().any():
        combined_data.loc[sensor_data.index, :] = impute_missing_values(sensor_data, 'Temperature', ['Humidity'])
    
    if sensor_data['Humidity'].isnull().any():
        combined_data.loc[sensor_data.index, :] = impute_missing_values(sensor_data, 'Humidity', ['Temperature'])

print("Data after imputing missing values:")
print(combined_data.isnull().sum())

/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UnixTime'] = df['Time'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)  # Convert Time to Unix timestamp
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: 

No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.


/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UnixTime'] = df['Time'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)  # Convert Time to Unix timestamp
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: 

No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.


/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UnixTime'] = df['Time'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)  # Convert Time to Unix timestamp
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: 

No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.


/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UnixTime'] = df['Time'].apply(lambda x: x.timestamp() if pd.notnull(x) else None)  # Convert Time to Unix timestamp
/var/folders/sd/6m81mt411jxfsqsc4d3ctrg40000gn/T/ipykernel_5016/2369181056.py:7: 

No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
No valid prediction data for Temperature.
No valid prediction data for Humidity.
Data after imputing missing values:
Time              0
Temperature    2235
Humidity       2235
Sensor Info       0
Sensor ID         0
dtype: int64


In [114]:
# # Fill missing values with the mean of the respective columns
# sample_dataset['Temperature'].fillna(sample_dataset['Temperature'].mean(), inplace=True)
# sample_dataset['Humidity'].fillna(sample_dataset['Humidity'].mean(), inplace=True)

# # Fit linear regression model for Temperature predicting Humidity
# X_temp = sample_dataset[['Temperature']].values
# y_hum = sample_dataset['Humidity'].values
# model_temp_hum = LinearRegression()
# model_temp_hum.fit(X_temp, y_hum)

# # Predict Humidity values and calculate residuals
# sample_dataset['Predicted Humidity'] = model_temp_hum.predict(X_temp)
# sample_dataset['Humidity Residuals'] = sample_dataset['Humidity'] - sample_dataset['Predicted Humidity']

# # Fit linear regression model for Humidity predicting Temperature
# X_hum = sample_dataset[['Humidity']].values
# y_temp = sample_dataset['Temperature'].values
# model_hum_temp = LinearRegression()
# model_hum_temp.fit(X_hum, y_temp)

# # Predict Temperature values and calculate residuals
# sample_dataset['Predicted Temperature'] = model_hum_temp.predict(X_hum)
# sample_dataset['Temperature Residuals'] = sample_dataset['Temperature'] - sample_dataset['Predicted Temperature']

# # Define thresholds for identifying outliers (e.g., 3 standard deviations from the mean of residuals)
# humidity_residuals_mean = sample_dataset['Humidity Residuals'].mean()
# humidity_residuals_std = sample_dataset['Humidity Residuals'].std()
# humidity_threshold = 3 * humidity_residuals_std

# temperature_residuals_mean = sample_dataset['Temperature Residuals'].mean()
# temperature_residuals_std = sample_dataset['Temperature Residuals'].std()
# temperature_threshold = 3 * temperature_residuals_std

# # Flag data points with residuals exceeding these thresholds as outliers
# sample_dataset['Humidity Outlier'] = np.abs(sample_dataset['Humidity Residuals']) > humidity_threshold
# sample_dataset['Temperature Outlier'] = np.abs(sample_dataset['Temperature Residuals']) > temperature_threshold

# # Combine outliers
# sample_dataset['Outlier LR'] = sample_dataset['Humidity Outlier'] | sample_dataset['Temperature Outlier']

# # 2. Isolation Forest
# iso_forest = IsolationForest(contamination=0.05, random_state=42)
# sample_dataset['Outlier IF'] = iso_forest.fit_predict(sample_dataset[['Temperature', 'Humidity']])
# sample_dataset['Outlier IF'] = sample_dataset['Outlier IF'].apply(lambda x: 'Outlier' if x == -1 else 'Inlier')

# # 3. One-Class SVM
# one_class_svm = OneClassSVM(nu=0.05, kernel='rbf', gamma=0.1)
# sample_dataset['Outlier SVM'] = one_class_svm.fit_predict(sample_dataset[['Temperature', 'Humidity']])
# sample_dataset['Outlier SVM'] = sample_dataset['Outlier SVM'].apply(lambda x: 'Outlier' if x == -1 else 'Inlier')

# # 4. Elliptic Envelope
# elliptic_env = EllipticEnvelope(contamination=0.05, random_state=42)
# sample_dataset['Outlier EE'] = elliptic_env.fit_predict(sample_dataset[['Temperature', 'Humidity']])
# sample_dataset['Outlier EE'] = sample_dataset['Outlier EE'].apply(lambda x: 'Outlier' if x == -1 else 'Inlier')

